In [61]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
data = pd.read_csv("PCAP3breath_001_labelled.csv",header=5)
df = data
df = df.drop('Frame', axis=1)
df = df.drop('Time (Seconds)', axis=1)
print(df)

             X         Y         Z       X.1       Y.1       Z.1       X.2  \
0     0.437758  0.425926  0.115648  0.413778  0.402811  0.159215  0.382547   
1     0.437774  0.425939  0.115683  0.413803  0.402817  0.159251  0.382567   
2     0.437773  0.425943  0.115711  0.413797  0.402814  0.159274  0.382572   
3     0.437782  0.425948  0.115734  0.413795  0.402817  0.159303  0.382563   
4     0.437785  0.425949  0.115747  0.413797  0.402806  0.159313  0.382556   
...        ...       ...       ...       ...       ...       ...       ...   
3186  0.425703  0.445132  0.115356  0.408023  0.418327  0.158387  0.379837   
3187  0.425700  0.445146  0.115338  0.408014  0.418364  0.158376  0.379804   
3188  0.425721  0.445147  0.115324  0.407999  0.418402  0.158372  0.379760   
3189  0.425742  0.445150  0.115306  0.407961  0.418443  0.158366  0.379685   
3190  0.425750  0.445154  0.115288  0.407937  0.418489  0.158362  0.379609   

           Y.2       Z.2       X.3  ...      Z.85      X.86    

In [44]:
df[df.columns[0]]

0       0.437758
1       0.437774
2       0.437773
3       0.437782
4       0.437785
          ...   
3186    0.425703
3187    0.425700
3188    0.425721
3189    0.425742
3190    0.425750
Name: X, Length: 3191, dtype: float64

In [63]:
def read_points(x,y,z):
    points = np.stack([x, y, z], axis=1)
    return points

class FPS:
    def __init__(self, points):
        self.points = np.unique(points, axis=0)

    def get_min_distance(self, a, b):
        distance = []
        for i in range(a.shape[0]):
            dis = np.sum(np.square(a[i] - b), axis=-1)
            distance.append(dis)
        distance = np.stack(distance, axis=-1)
        distance = np.min(distance, axis=-1)
        return np.argmax(distance)
    @staticmethod
    def get_model_corners(model):
        min_x, max_x = np.min(model[:, 0]), np.max(model[:, 0])
        min_y, max_y = np.min(model[:, 1]), np.max(model[:, 1])
        min_z, max_z = np.min(model[:, 2]), np.max(model[:, 2])
        corners_3d = np.array([
            [min_x, min_y, min_z],
            [min_x, min_y, max_z],
            [min_x, max_y, min_z],
            [min_x, max_y, max_z],
            [max_x, min_y, min_z],
            [max_x, min_y, max_z],
            [max_x, max_y, min_z],
            [max_x, max_y, max_z],
        ])
        return corners_3d
    def compute_fps(self, K):
        # compute centre point location
        corner_3d = self.get_model_corners(self.points)
        center_3d = (np.max(corner_3d, 0) + np.min(corner_3d, 0)) / 2
        A = np.array([center_3d])
        B = np.array(self.points)
        t = []
        # looking for k nodes
        for i in range(K-1):
            # print(i)
            max_id = self.get_min_distance(A, B)
            A = np.append(A, np.array([B[max_id]]), 0)
            B = np.delete(B, max_id, 0)
            t.append(max_id)
        print(len(A))
        return A 
        # file.writelines("ply\n")
        # file.writelines("format ascii 1.0\n")
        # file.writelines("element vertex "+str(K+1)+"\n")
        # file.writelines("property float x\n")
        # file.writelines("property float y\n")
        # file.writelines("property float z\n")
        # file.writelines("element face 0\n")
        # file.writelines("property list uchar int vertex_indices\n")
        # file.writelines("end_header\n")
file=open('FPSPoints.txt','w')
count = 1
for i in range(2,268,3):
  f = FPS(read_points(df[df.columns[i-2]],df[df.columns[i-1]],df[df.columns[i]]))
  C = f.compute_fps(100)#Number of sampling points
  file.writelines("marker"+str(count)+"\n")
  count = count + 1
  for j in C:
    file.writelines(str(float(j[0]))+"\t"+str(float(j[1]))+"\t"+str(float(j[2]))+"\n")

100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100
100


In [38]:
# !pip install open3d
# !pip3 install open3d-python
# import open3d as o3d
# from open3d import *

# print("load a ply point cloud")
# pcd = o3d.io.read_point_cloud("fps_of.ply")
# points = np.asarray(pcd.points)
# pcd.paint_uniform_color([0, 0, 1])#指定显示为蓝色
#     #点云显示
# o3d.visualization.draw_geometries([pcd],#待显示的点云列表
#                                       window_name="点云显示",
#                                       point_show_normal=False,
#                                       width=800,  # 窗口宽度
#                                       height=600)  # 窗口高度

load a ply point cloud
[Open3D WARNING] GLFW Error: X11: The DISPLAY environment variable is missing
[Open3D WARNING] Failed to initialize GLFW
[Open3D WARNING] [DrawGeometries] Failed creating OpenGL window.


In [ ]:
data2 = pd.read_csv("PCAP3breath_007_labelled.csv",header=5)
print(data2)

      Frame  Time (Seconds)         X         Y         Z       X.1       Y.1  \
0         6        0.000000  0.494002  0.444577  0.155395  0.465330  0.414194   
1         7        0.008333  0.493937  0.444417  0.155267  0.465395  0.414012   
2         8        0.016667  0.493901  0.444278  0.155150  0.465445  0.413848   
3         9        0.025000  0.493845  0.444117  0.155030  0.465567  0.413679   
4        10        0.033333  0.493815  0.443950  0.154903  0.465639  0.413508   
...     ...             ...       ...       ...       ...       ...       ...   
4872   4878       40.600000  0.434933  0.432674  0.094103  0.414108  0.407709   
4873   4879       40.608333  0.434945  0.432675  0.094182  0.414110  0.407710   
4874   4880       40.616667  0.434977  0.432665  0.094264  0.414152  0.407697   
4875   4881       40.625000  0.435031  0.432650  0.094344  0.414220  0.407686   
4876   4882       40.633333  0.435085  0.432624  0.094425  0.414278  0.407656   

           Z.1       X.2   

In [ ]:
data3 = pd.read_csv("3BR Rest_002_labelled.csv",header=5)
print(data3)

      Frame  Time (Seconds)         X         Y         Z       X.1       Y.1  \
0         0        0.000000 -0.114362  0.337908  0.307746 -0.155794  0.316392   
1         1        0.008333 -0.114368  0.337905  0.307730 -0.155798  0.316380   
2         2        0.016667 -0.114374  0.337890  0.307675 -0.155807  0.316355   
3         3        0.025000 -0.114376  0.337872  0.307633 -0.155816  0.316329   
4         4        0.033333 -0.114376  0.337868  0.307606 -0.155824  0.316310   
...     ...             ...       ...       ...       ...       ...       ...   
2607   2607       21.725000 -0.095715  0.351029  0.267069 -0.145424  0.324023   
2608   2608       21.733333 -0.095761  0.351017  0.266821 -0.145487  0.324029   
2609   2609       21.741667 -0.095793  0.351011  0.266596 -0.145549  0.324041   
2610   2610       21.750000 -0.095836  0.351025  0.266387 -0.145604  0.324064   
2611   2611       21.758333 -0.095872  0.351039  0.266170 -0.145665  0.324095   

           Z.1       X.2   

In [ ]:
data4 = pd.read_csv("3BR Rest_006_labelled.csv",header=5)
print(data4)

      Frame  Time (Seconds)         X         Y         Z       X.1       Y.1  \
0         0        0.000000 -0.125895  0.332505  0.298585 -0.166842  0.305760   
1         1        0.008333 -0.125889  0.332509  0.298583 -0.166832  0.305771   
2         2        0.016667 -0.125878  0.332532  0.298560 -0.166824  0.305794   
3         3        0.025000 -0.125866  0.332546  0.298532 -0.166817  0.305818   
4         4        0.033333 -0.125861  0.332568  0.298515 -0.166814  0.305835   
...     ...             ...       ...       ...       ...       ...       ...   
3494   3494       29.116667 -0.111436  0.361766  0.294797 -0.151527  0.330172   
3495   3495       29.125000 -0.111481  0.361763  0.294774 -0.151563  0.330160   
3496   3496       29.133333 -0.111518  0.361750  0.294759 -0.151590  0.330153   
3497   3497       29.141667 -0.111560  0.361743  0.294751 -0.151627  0.330134   
3498   3498       29.150000 -0.111597  0.361729  0.294722 -0.151659  0.330113   

           Z.1       X.2   